# Widgets

> Collection of different widgets for Jupyter Notebook


In [1]:
# | default_exp tools.widgets

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
# | export
# | hide

from __future__ import annotations

import datetime

# sys and paths imports
import json
import logging
import math
import os
import re
import sys
import tkinter as tk
import warnings

# typing imports
from abc import ABC, abstractmethod
from collections import Counter, defaultdict
from dataclasses import dataclass
from datetime import datetime
from enum import Enum
from os import PathLike
from pathlib import Path
from random import randint
from time import sleep
from tkinter import filedialog
from typing import Any, Callable, ClassVar, Generic, Iterable, Literal, TypeVar

# cv and image imports
import cv2
import geopy
import imageio
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from bidict import bidict
from dotenv import load_dotenv
from geopy.exc import GeocoderServiceError, GeocoderTimedOut
from geopy.geocoders import Nominatim

# widgets imports
from IPython.display import clear_output, display
from ipywidgets import Accordion, Button, Layout, VBox
from matplotlib.ticker import PercentFormatter
from PIL import Image as PImage
from pymemcache.client import base
from pyproj import Geod
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import LineString
from torch import layout
from tqdm.notebook import tqdm
from typing import Callable
from numpy import dtype
from ds_contrib.core.utils import Iterifiable, listify

from ds_contrib.core.data.video import (
    IFramesSampler,
    get_video_metadata,
    sample_frames_from_video,
)
from ds_contrib.core.files.structure import FSNode, GSBrowserFileStructure
from ds_contrib.core.paths import (
    Directory,
    PathLike,
    handle_existing,
    list_paths,
    pathify,
)

# visualization imports
from ds_contrib.core.plotting import Color, plot
from ds_contrib.core.utils import (
    Iterifiable,
    T,
    dictify_with_names,
    exclusive_args,
    listify,
)
from ds_contrib.tools.io.gscloud import GSBrowser, GSBrowserContext

In [4]:
# | export
# | hide

os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd

In [5]:
logger = logging.getLogger(__name__)

In [6]:
# | export
# | hide


class IWidget(ABC):
    def __init__(self) -> None:
        self._observers: dict[str, list[Callable]] = defaultdict(list)
        self._setup_gui()
        self._init_state_machine()
        self._attach_callbacks()

    @abstractmethod
    def _setup_gui(self):
        raise NotImplementedError

    def _attach_callbacks(self):
        pass

    def _init_state_machine(self):
        pass

    @abstractmethod
    def widgets(self, isolated=True) -> widgets.Widget | dict[str, widgets.Widget]:
        raise NotImplementedError

## Pagination


In [7]:
# | export


class PaginationIterator(Generic[T]):
    @exclusive_args(["start_index", "start_key"])
    def __init__(
        self,
        keys_collection: list[T] | pd.Index,
        page_size: int,
        start_index: int | None = None,
        start_key: T | None = None,
        alignment: Literal["center", "left", "right"] = "center",
    ):
        """Circular pagination for a collection of a given size and a page size

        Parameters
        ----------
        page_size : int
            Size of the range to show around the current index
        collection_size : int
            Size of the collection
        index : int, optional
            Initial index, by default 0
        """
        self._ordered_collection = (
            pd.Index(keys_collection)
            if isinstance(keys_collection, list)
            else keys_collection
        )
        self._collection_size = len(self._ordered_collection)
        self._page_size = page_size
        self._index = self._init_start_index(start_index, start_key)
        self._alignment = alignment
        self._observers = []

    def _init_start_index(self, start_index: int | None, start_key: T | None):
        if start_index is not None:
            return start_index
        elif start_key is not None:
            return self._ordered_collection.get_loc(start_key)
        else:
            return 0

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}: (collection_size={self._collection_size}, page_size={self._page_size}, index={self.current_index}, current_key={self.current_key}"

    @property
    def current_index(self) -> int:
        return self._index

    def __len__(self):
        return self._collection_size

    @property
    def current_key(self) -> T:
        return self._ordered_collection[self._index]

    @current_index.setter
    def current_index(self, value):
        self._index = self.index_mapper(value)
        self._notify_index_changed()

    def index_mapper(self, value):
        return min(max(0, value), self._collection_size - 1)

    @current_key.setter
    def current_key(self, key: T):
        self.current_index = self._ordered_collection.get_loc(key)

    @property
    def page_keys(self) -> Iterable[T]:
        return self._ordered_collection[self.page_inds_slice].values

    @property
    def page_inds_slice(self):
        if self._alignment == "center":
            half_size = self._page_size // 2
            if self.current_index < half_size:
                left = 0
                right = min(self._collection_size, self._page_size)
            elif self.current_index > self._collection_size - half_size:
                left = max(0, self._collection_size - self._page_size)
                right = self._collection_size
            else:
                left = self.current_index - half_size
                right = self.current_index + (self._page_size - half_size)
        elif self._alignment == "left":
            left = self.current_index
            right = min(self._collection_size, self.current_index + self._page_size)
        elif self._alignment == "right":
            left = max(0, self.current_index - self._page_size)
            right = self.current_index
        else:
            raise ValueError(f"Invalid alignment: {self._alignment}")
        return slice(left, right)

    @property
    def page_inds(self) -> range:
        slize = self.page_inds_slice
        return range(slize.start, slize.stop)

    @property
    def page_keys_slice(self):
        stop = (
            None
            if self.page_inds_slice.stop == self._collection_size
            else self._ordered_collection[self.page_inds_slice.stop]
        )
        return slice(self._ordered_collection[self.page_inds_slice.start], stop)

    def forward(self, step=1):
        self.current_index += step

    def backward(self, step=1):
        self.current_index -= step

    def has_next_page(self, step_size=1, unique=False):
        """Check if there is a next page

        Parameters
        ----------
        step_size : int, optional
            check if there is a next page after the current page + step_size, by default 1
        unique : bool, optional
            whether to check if the next page is unique, by default False,
            unique means that the next page has at least one key that is not in the current page

        Returns
        -------
        bool
            whether there is a next page
        """
        if unique:
            return (
                self.index_mapper(self.current_index + step_size)
                > self.page_inds_slice.stop
            )
        else:
            return self.current_index + 1 < self._collection_size

    def has_previous_page(self, step_size=1, unique=False):
        if unique:
            return (
                self.index_mapper(self.current_index - step_size)
                < self.page_inds_slice.start
            )
        else:
            return self.current_index > 0

    def link_to(
        self,
        other: PaginationIterator[T],
        direct_mapping_func: Callable[[T], T] | None = None,
        reverse_mapping_func: Callable[[T], T] | None = None,
        bidirectional: bool = False,
    ):
        """Link two pagination iterators together

        The linking is based on the keys of the iterators, so when the key of one iterator changes, the other one will change as well
        The key conversion functions are used to convert the keys of one iterator to the keys of the other iterator if necessary

        Parameters
        ----------
        other : PaginationIterator[T]
            other iterator to link to, when other iterator changes, this one will change as well
        direct_mapping_func : Callable[[T], T] | None, optional
            how to convert the key of this iterator to the key of the other iterator, by default None
                if None, the identity function is used
        reverse_mapping_func : Callable[[T], T] | None, optional
            how to convert the key of the other iterator to the key of this iterator, by default None
                if None, the identity function is used
        bidirectional : bool, optional
            whether to link the other iterator to this one as well, by default False
        """
        if direct_mapping_func is None:
            direct_mapping_func = lambda x: x
        if reverse_mapping_func is None:
            reverse_mapping_func = lambda x: x

        other.observe_index_changed(
            lambda _: self.move_to(key=reverse_mapping_func(other.current_key))
        )
        if bidirectional:
            self.observe_index_changed(
                lambda _: other.move_to(key=direct_mapping_func(self.current_key))
            )

    @exclusive_args(["index", "key"], may_be_empty=True)
    def move_to(self, index: int | None = None, key: T | None = None):
        """Moves the iterator to the given index or key

        Parameters
        ----------
        index : int | None, optional
            index (serial number) to move to, by default None, exclusive with key
        key : T | None, optional
            key to move to, by default None, exclusive with index

        if both index and key are None, nothing happens
        """
        if index is not None:
            self.current_index = index
        elif key is not None:
            self.current_key = key
        else:
            logger.debug(
                "PaginationIterator.move_to: no index or key provided, doing nothing"
            )

    def observe_index_changed(
        self, observer_cb: Callable[[CircularPaginationIterator[T]], Any]
    ):
        """Add an observer to the index change event of this iterator

        If the index of the iterator changes, the observer will be called with the iterator as an argument

        Parameters
        ----------
        observer_cb : Callable[[CircularPaginationIterator[T]], Any]
            callback to call when the index of the iterator changes, this iterator will be passed as an argument
        """
        self._observers.append(observer_cb)

    def _notify_index_changed(self):
        for observer_cb in self._observers:
            observer_cb(self)


class CircularPaginationIterator(PaginationIterator):
    def index_mapper(self, index):
        return index % self._collection_size

#### Pagination examples


In [8]:
p = PaginationIterator(list("abcdefgh"), 3, start_key="c", alignment="left")

In [9]:
print(p.current_index, p.current_key, p.page_inds_slice)
p.forward()

2 c slice(2, 5, None)


Circular pagination is used to browse objects in a loop.


In [10]:
cp = CircularPaginationIterator(list("abcdefgh"), 3, start_key="c")

In [11]:
cp, cp.page_inds_slice, cp.page_keys_slice, cp.page_inds, cp.page_keys

(CircularPaginationIterator: (collection_size=8, page_size=3, index=2, current_key=c,
 slice(1, 4, None),
 slice('b', 'e', None),
 range(1, 4),
 array(['b', 'c', 'd'], dtype=object))

In [12]:
print(f"before move: {cp.current_index}, {cp.current_key}")
cp.forward()
print(f"after move: {cp.current_index}, {cp.current_key}")

before move: 2, c
after move: 3, d


Link two paginations with each other by using observer


In [13]:
cp = CircularPaginationIterator(list("abcdefgh"), 3, start_key="c")
cp_2 = CircularPaginationIterator(list("qwerty"), 4, start_index=5)

In [14]:
mapping = {"a": "q", "b": "w", "c": "e", "d": "r", "e": "t", "f": "y"}
cp_2.link_to(
    cp, reverse_mapping_func=lambda x: mapping.get(x, None), bidirectional=False
)

In [15]:
print("Initial state:", cp.current_key, cp_2.current_key)
cp.forward()
print(
    "After `cp.forward()`, `cp2` mapped key to `cp`:", cp.current_key, cp_2.current_key
)
cp.forward(3)
print(
    "After move to 3 items forward, `cp2` hasnot found a mapping, so it stays at the same key:",
    cp.current_key,
    cp_2.current_key,
)
cp.forward(2)
print(
    "After move to 2 items forward due to circular iteration we returned to the key we can map:",
    cp.current_key,
    cp_2.current_key,
)

Initial state: c y
After `cp.forward()`, `cp2` mapped key to `cp`: d r
After move to 3 items forward, `cp2` hasnot found a mapping, so it stays at the same key: g r
After move to 2 items forward due to circular iteration we returned to the key we can map: a q


#### Pagination widget


In [16]:
# | export


class PaginationWidget(IWidget, Generic[T]):
    def __init__(
        self,
        pagination: PaginationIterator[T],
        step_sizes: Iterifiable[int] = 1,
    ) -> None:
        self._pagination = pagination
        self._step_sizes: list[int] = listify(step_sizes)  # type: ignore
        self._step_size: int = self._step_sizes[0]

        super().__init__()

    def _setup_gui(self):
        # self.status_bar = StatusBar(self._statuses)
        self.prev_button = widgets.Button(description=f"Prev")
        self.next_button = widgets.Button(description=f"Next")
        self.position_field = widgets.HTML()

        self._has_step_size_selector = len(self._step_sizes) > 1
        if self._has_step_size_selector:
            self.step_size_selector = widgets.Dropdown(
                options=self._step_sizes,
                value=self._step_sizes[0],
                description="Step size: ",
                dtype=int,
            )

    def _attach_callbacks(self):
        self.prev_button.on_click(self.cb_btn_backward)
        self.next_button.on_click(self.cb_btn_forward)

        # Observers
        if self._has_step_size_selector:
            self.step_size_selector.observe(self.cb_on_dropdown_change, names="value")
        self._pagination.observe_index_changed(self._cb_on_pagination_change)

    @property
    def index(self) -> int:
        return self._pagination.current_index

    def __len__(self):
        return len(self._pagination)

    @property
    def current_key(self) -> T:
        return self._pagination.current_key

    @property
    def step_size(self) -> int:
        return self._step_size

    def _cb_on_pagination_change(self, *args):
        has_next_page = self._pagination.has_next_page(self.step_size)
        has_prev_page = self._pagination.has_previous_page(self.step_size)
        self.next_button.disabled = not has_next_page
        self.prev_button.disabled = not has_prev_page
        self.position_field.value = f"{self._pagination.current_index+1}/{len(self._pagination)} | Page: [{self._pagination.page_inds_slice.start+1}:{self._pagination.page_inds_slice.stop}] <br>"
        self.prev_button.description = (
            f"[{self._pagination.index_mapper(self._pagination.current_index - self._step_size) + 1}] Prev"
            if has_prev_page
            else "Boundary"
        )
        self.next_button.description = (
            f"Next [{self._pagination.index_mapper(self._pagination.current_index + self._step_size)+1}]"
            if has_next_page
            else "Boundary"
        )

    def cb_btn_forward(self, btn=None):
        self._pagination.forward(self.step_size)

    def cb_btn_backward(self, btn=None):
        self._pagination.backward(self.step_size)

    def cb_on_dropdown_change(self, change):
        """Callback function to handle changes in the dropdown value."""
        if self._has_step_size_selector:
            self._step_size = change.new
            self._cb_on_pagination_change()
        else:
            raise ValueError("No step size selector widget")

    def widgets(self, isolated=True) -> widgets.HBox | dict[str, widgets.Widget]:
        # init widgets
        self._cb_on_pagination_change()

        children = {
            "position": self.position_field,
            "prev": self.prev_button,
            "next": self.next_button,
        }
        if self._has_step_size_selector:
            children["step_size_selector"] = self.step_size_selector
        if isolated:
            return widgets.HBox(
                tuple(children.values()),
                layout=widgets.Layout(justify_content="center"),
            )
        else:
            return children

In [17]:
collection = "abcdefghklmnqwertyuiop"
pagination = PaginationIterator(collection, 5, start_index=0)
widget = PaginationWidget(pagination, [1, 3, 5]).widgets(isolated=True)

In [18]:
# example of observer callback that prints value
pagination.observe_index_changed(lambda p: print(p))

In [19]:
# | eval: false

display(widget)

## Status provider


In [20]:
# | export


class IStatus:
    pass


class QAStatus(IStatus, Enum):
    """Status of an item in the browser

    Parameters
    ----------
    Enum :
        NOT_VIEWED : 0
        ADDED : 1
        REMOVED : 2
        VIEWED : 3
    """

    NOT_VIEWED = 0
    ADDED = 1
    REMOVED = 2
    VIEWED = 3

    def __str__(self):
        return self.name

    @property
    def infinitive(self) -> str:
        if self == QAStatus.NOT_VIEWED:
            return "unview"
        elif self == QAStatus.ADDED:
            return "add"
        elif self == QAStatus.REMOVED:
            return "remove"
        elif self == QAStatus.VIEWED:
            return "view"

    @property
    def color(self):
        """Returns the color of the item in the browser

        Returns
        -------
        str
            str representation of color of the item in the browser (conforms css)
        """
        if self == QAStatus.NOT_VIEWED:
            return "grey"
        elif self == QAStatus.ADDED:
            return "green"
        elif self == QAStatus.REMOVED:
            return "red"
        elif self == QAStatus.VIEWED:
            return "lightgrey"


class IStatusProvider(Generic[T]):
    def __init__(self, status_set: list[IStatus]) -> None:
        self._observers: list[Callable[[T, IStatus], Any]] = []
        self._status_set = status_set

    @abstractmethod
    def get_status(self, key: T) -> IStatus:
        raise NotImplementedError

    def get_statuses(self, keys: Iterifiable[T]) -> Iterable[IStatus]:
        for k in listify(keys):
            yield self.get_status(k)

    def __len__(self) -> int:
        raise NotImplementedError

    def stats(self) -> Counter[IStatus]:
        return Counter(self.get_statuses(self.keys()))

    def set_status(self, key: T, status: IStatus):
        self._set_status(key, status)
        self._notify_status_changed(key, status)

    def observe_on_status_change(self, observer_cb: Callable[[T, IStatus], Any]):
        self._observers.append(observer_cb)

    def _notify_status_changed(self, key: T, status: IStatus):
        for observer_cb in self._observers:
            observer_cb(key, status)

    @abstractmethod
    def _set_status(self, key: T, status: IStatus):
        raise NotImplementedError

    @property
    def status_set(self) -> list[IStatus]:
        return self._status_set


class DictStatusProvider(IStatusProvider[T]):
    def __init__(self, statuses: dict[T, IStatus], status_set: list[IStatus]) -> None:
        super().__init__(status_set)
        self._df = statuses

    def get_status(self, key: T) -> IStatus:
        return self._df[key]

    def _set_status(self, key: T, status: IStatus):
        self._df[key] = status

    def stats(self) -> Counter[IStatus]:
        return Counter(self._df.values())

    def __len__(self) -> int:
        return len(self._df)

    def __repr__(self) -> str:
        return self._df.__repr__()

In [21]:
# | hide
# TODO: add examples

## Status bar


In [55]:
# | export


class StatusBar(IWidget, Generic[T]):
    """
    Status bar object containing widget and storing states of statuses
    """

    def __init__(
        self,
        statuses: IStatusProvider[T],
        pagination: CircularPaginationIterator[T],
        status_style: dict[IStatus, str],
    ) -> None:
        """Returns new status bar object

        Parameters
        ----------
        statuses : list[BrowseStatus]
            List of statuses for each image in the browser
        """
        self._status_provider = statuses
        self._pagination = pagination
        self._status_style = status_style

        self._style = {
            "width": "4px",
            "height": "12px",
            "border": "2px solid blue",
        }
        super().__init__()

    def _setup_gui(self):
        self._status_bar_widget = widgets.HTML(
            value="",
            layout=widgets.Layout(width="90%"),
        )

    def _attach_callbacks(self):
        self._pagination.observe_index_changed(self._generate_status_bar)
        self._status_provider.observe_on_status_change(self._generate_status_bar)

    def _generate_single_status_item(self, status: IStatus, selected=False) -> str:
        color = self._status_style[status]
        border = self._style["border"] if selected else "none"
        block = f'<span style="display: inline-block; width: {self._style["width"]}; height: {self._style["height"]}; background-color: {color}; border: {border}; margin: 1px;"></span>'
        return block

    def _generate_left_arrow(self) -> str:
        """Generate a left-pointing arrow."""
        return f"""
        <div style="
            width: 0;
            height: 0;
            border-top: 5px solid transparent;
            border-bottom: 5px solid transparent;
            border-right: 10px solid blue;
            display: inline-block;
            vertical-align: center;
            margin: 10px;
        "></div>
        """

    def _generate_right_arrow(self) -> str:
        """Generate a right-pointing arrow."""
        return f"""
        <div style="
            width: 0;
            height: 0;
            border-top: 5px solid transparent;
            border-bottom: 5px solid transparent;
            border-left: 10px solid blue;
            display: inline-block;
            vertical-align: center;
            margin: 10px;
        "></div>
        """

    def _generate_status_bar(self, *args):
        page = self._pagination.page_inds_slice
        left, right = page.start, page.stop
        left_arrow = "" if left == 0 else self._generate_left_arrow()
        right_arrow = (
            "" if right == len(self._status_provider) else self._generate_right_arrow()
        )
        visual_blocks = [
            self._generate_single_status_item(
                self._status_provider.get_status(key),
                selected=(ind == self._pagination.current_index),
            )
            for ind, key in zip(range(left, right), self._pagination.page_keys)
        ]
        self._status_bar_widget.value = f"""
        <div style="display: flex; align-items: center;">
            {left_arrow}
            <div style="flex;">
                {''.join(visual_blocks)}
            </div>
            {right_arrow}
        </div>
        """

    def widgets(self, isolated=True) -> widgets.HBox | dict[str, widgets.Widget]:
        # Init widgets
        self._generate_status_bar(self._pagination)

        # Pack widgets
        children = {"status_bar": self._status_bar_widget}
        if isolated:
            return widgets.HBox(
                tuple(children.values()),
            )
        else:
            return children

#### Status bar example


In [56]:
statuses = DictStatusProvider(
    {
        "a": QAStatus.ADDED,
        "b": QAStatus.REMOVED,
        "c": QAStatus.VIEWED,
        "d": QAStatus.NOT_VIEWED,
        "e": QAStatus.ADDED,
        "f": QAStatus.ADDED,
        "g": QAStatus.NOT_VIEWED,
        "h": QAStatus.VIEWED,
    },
    list(QAStatus),
)

pagination = CircularPaginationIterator(
    list("abcdefgh"), 10, start_index=3, alignment="center"
)

status_style = {
    QAStatus.ADDED: QAStatus.ADDED.color,
    QAStatus.REMOVED: QAStatus.REMOVED.color,
    QAStatus.VIEWED: QAStatus.VIEWED.color,
    QAStatus.NOT_VIEWED: QAStatus.NOT_VIEWED.color,
}

In [57]:
sb = StatusBar(statuses, pagination, status_style)

In [58]:
# | eval: false

display(sb.widgets(isolated=True))

In [59]:
pagination.forward()

## Multi-select widget


In [27]:
# | export


class CheckboxButton(Button):
    def __init__(self, *args, selected=False, short_description=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.selected = selected
        self.short_description = (
            short_description if short_description else self.description
        )
        self.icon = "check-square-o" if selected else "square-o"

    def change_selection(self):
        self.selected = not self.selected
        self.icon = "check-square-o" if self.selected else "square-o"


class MultiCheckBoxWidget(IWidget):
    def __init__(
        self,
        checkbox_names: list[str],
        hints: list[str] | None = None,
        short_descriptions: list[str] | None = None,
    ):
        self.checkbox_names = checkbox_names
        self.hints = hints if hints else checkbox_names
        self.short_descriptions = (
            short_descriptions if short_descriptions else checkbox_names
        )
        super().__init__()

    def _setup_gui(self):
        self.checkboxes = [
            CheckboxButton(
                description=name,
                layout=Layout(width="auto"),
                tooltip=f"{self.hints[i]}",
                short_description=self.short_descriptions[i],
            )
            for i, name in enumerate(self.checkbox_names)
        ]
        # Set the first checkbox to be selected by default
        self.checkboxes[0].change_selection()
        self.checkbox_container = VBox(self.checkboxes)
        self.accordion = Accordion(
            children=[self.checkbox_container], layout=Layout(width="20%")
        )
        self._update_title()

    def _attach_callbacks(self):
        for checkbox in self.checkboxes:
            checkbox.on_click(self._toggle_checkbox)

    def _toggle_checkbox(self, change):
        selected_checkboxes = [
            checkbox for checkbox in self.checkboxes if checkbox.selected
        ]
        if len(selected_checkboxes) > 1 or not change.selected:
            change.change_selection()
            self._update_title()
        else:
            # Prevent the last selected checkbox from being deselected
            change.selected = True
        self._notify_checkbox_change(self.selected)

    def _update_title(self):
        selected = [
            checkbox.short_description
            for checkbox in self.checkboxes
            if checkbox.selected
        ]
        if selected:
            self.accordion.set_title(0, f"Selected: {', '.join(selected)}")
        else:
            self.accordion.set_title(0, "No Selection")

    @property
    def selected(self):
        return [
            checkbox.description for checkbox in self.checkboxes if checkbox.selected
        ]

    def observe_on_checkbox_change(self, observer_cb: Callable[[list[str]], Any]):
        self._observers["checkbox_change"].append(observer_cb)

    def _notify_checkbox_change(self, chosen_checkboxes: list[str]):
        for observer_cb in self._observers["checkbox_change"]:
            observer_cb(chosen_checkboxes)

    def widgets(self, isolated=True):
        if isolated:
            return self.accordion
        else:
            return {"accordion": self.accordion}

#### Example


In [28]:
# Usage
widget = MultiCheckBoxWidget(
    ["Protocol 3", "Protocol 6", "Validation 1", "Validation 2"],
    short_descriptions=["P3", "P6", "V1", "V2"],
)
display(widget.widgets(isolated=True))

Accordion(children=(VBox(children=(CheckboxButton(description='Protocol 3', icon='check-square-o', layout=Layo…

## Change status widget


In [29]:
# | export


# TODO[High](Feature): Add MultiCheckboxWidget instead of dropdown for destination selection
class ChangeStatusWidget(IWidget):
    def __init__(
        self,
        status_providers: dict[str, IStatusProvider[T]] | IStatusProvider[T],
        iterator: CircularPaginationIterator[T],
        visited_status: tuple[IStatus, IStatus],
    ) -> None:
        self._status_providers: dict[str, IStatusProvider] = dictify_with_names(
            status_providers, "status_provider"
        )

        self._iterator = iterator
        # TODO: extend to multiple status providers
        self._available_statuses = [
            status
            for status in list(self._status_providers.values())[0].status_set
            if status != visited_status[1]
        ]

        self._visited_status = visited_status
        self._destinations = list(self._status_providers.keys())
        self._has_different_destinations = len(self._destinations) > 1

        self._current_status = None
        self._current_key = self._iterator.current_key
        self._current_destination = self._destinations[0]

        super().__init__()
        self._get_status(iterator.current_key)

    def _setup_gui(self):
        self._status_selector = widgets.Dropdown(
            options=self._available_statuses,
            description="Select:",
        )
        if self._has_different_destinations:
            self._destination_selector = widgets.Dropdown(
                options=self._destinations,
                description="Destination: ",
            )
        self._change_status_button = widgets.Button(description="Change status")
        self._status_view = widgets.HTML(value="")
        self._status_stats = widgets.HTML(value="")

    def _attach_callbacks(self):
        self._status_providers[self._current_destination].observe_on_status_change(
            self._cb_on_status_change
        )
        self._iterator.observe_index_changed(self._cb_on_index_change)
        self._status_selector.observe(self._cb_on_status_selector_change, names="value")
        if self._has_different_destinations:
            self._destination_selector.observe(
                self._cb_on_destination_selector_change, names="value"
            )
        self._change_status_button.on_click(self._cb_on_change_status_button)

    def available_statuses(self, status: QAStatus):
        return [s for s in self._available_statuses if s != status]

    def _cb_on_destination_selector_change(self, change):
        self._current_destination = change.new
        self._get_status(self._current_key)

    def _get_status(self, key: T):
        self._current_status = self._status_providers[
            self._current_destination
        ].get_status(key)

        # update buttons
        self._status_view.value = f"Status: {self._current_status}"
        self._status_selector.options = self.available_statuses(self._current_status)
        self._update_change_status_button(self._status_selector.value)
        self._update_stats(key, self._current_status)

    def _update_change_status_button(self, status: QAStatus):
        self._change_status_button.description = status.infinitive.capitalize()
        self._change_status_button.value = status
        self._change_status_button.disabled = self._current_status == status
        self._change_status_button.style.button_color = status.color

    def _set_status(self, key: T, status: QAStatus):
        self._status_providers[self._current_destination].set_status(key, status)
        self._current_status = status

    def _update_stats(self, key, status: IStatus):
        self._status_stats.value = self.status_stats_str()

    def _cb_on_status_change(self, key: T, status: IStatus):
        if key == self._iterator.current_key:
            self._get_status(key)
        else:
            self._update_stats(key, status)

    def _cb_on_index_change(self, iterator: CircularPaginationIterator[T]):
        if self._current_status == self._visited_status[0]:
            self._set_status(self._current_key, self._visited_status[1])
        self._current_key = iterator.current_key
        self._get_status(self._current_key)

    def _cb_on_change_status_button(self, btn=None):
        self._set_status(self._iterator.current_key, self._change_status_button.value)
        self._iterator.forward()

    def _cb_on_status_selector_change(self, change):
        self._update_change_status_button(change.new)

    def status_stats_str(self) -> str:
        """Returns string with statistics of statuses

        Returns
        -------
        str
            String with statistics of statuses
        """
        stats = self._status_providers[self._current_destination].stats()
        overall = len(self._status_providers[self._current_destination])
        s = "Statistics: "
        for status, number in stats.items():
            s += f"{status}: {number} ({number/overall if overall > 0 else 0:.2%}), "
        return s[:-2]

    def widgets(self, isolated=True) -> widgets.HBox | dict[str, widgets.Widget]:
        if isolated:
            bottom_box_children = [self._change_status_button, self._status_selector]
            if self._has_different_destinations:
                bottom_box_children.append(self._destination_selector)

            return widgets.VBox(
                [
                    widgets.HBox(
                        [self._status_view, self._status_stats],
                        layout=widgets.Layout(
                            justify_content="space-between", width="80%"
                        ),
                    ),
                    widgets.HBox(bottom_box_children),
                ]
            )
        else:
            d = {
                "status_view": self._status_view,
                "status_stats": self._status_stats,
                "change_status_button": self._change_status_button,
                "status_selector": self._status_selector,
            }
            if self._has_different_destinations:
                d["destination_selector"] = self._destination_selector
            return d

#### Example: ChangeStatusWidget


In [30]:
status_provider_1 = DictStatusProvider(
    {
        "a": QAStatus.NOT_VIEWED,
        "b": QAStatus.REMOVED,
        "c": QAStatus.VIEWED,
        "d": QAStatus.NOT_VIEWED,
        "e": QAStatus.ADDED,
        "f": QAStatus.ADDED,
        "g": QAStatus.NOT_VIEWED,
        "h": QAStatus.VIEWED,
    },
    list(QAStatus),
)
status_provider_2 = DictStatusProvider(
    {
        "a": QAStatus.NOT_VIEWED,
        "b": QAStatus.NOT_VIEWED,
        "c": QAStatus.NOT_VIEWED,
        "d": QAStatus.NOT_VIEWED,
        "e": QAStatus.NOT_VIEWED,
        "f": QAStatus.NOT_VIEWED,
        "g": QAStatus.NOT_VIEWED,
        "h": QAStatus.NOT_VIEWED,
    },
    list(QAStatus),
)

pagination = CircularPaginationIterator(list("abcdefgh"), 10, start_index=0)

sb_1 = StatusBar(status_provider_1, pagination, status_style)
sb_2 = StatusBar(status_provider_2, pagination, status_style)

In [31]:
change_status_widget = ChangeStatusWidget(
    [status_provider_1, status_provider_2],
    pagination,
    visited_status=(QAStatus.NOT_VIEWED, QAStatus.VIEWED),
)

In [32]:
display(sb_1.widgets(isolated=True))
display(sb_2.widgets(isolated=True))
display(change_status_widget.widgets(isolated=True))

In [33]:
pagination.forward()

## Image browser


In [34]:
# | export


class ImageBrowser(IWidget):
    def __init__(
        self,
        image_paths: list[PathLike],
        status_provider: IStatusProvider[T],
        start_index: int = 0,
        status_bar_page_size: int = 100,
        step_sizes: Iterifiable[int] = (1, 5, 10, 100, 1000),
    ) -> None:
        self._image_paths = [Path(p).absolute().resolve() for p in image_paths]
        self._pagination = PaginationIterator(
            image_paths, status_bar_page_size, start_index=start_index
        )
        self._status_provider = status_provider

        self._step_sizes: list[int] = listify(step_sizes)  # type: ignore
        super().__init__()

    def _setup_gui(self):
        self.change_status_widget = ChangeStatusWidget(
            self._status_provider,
            self._pagination,
            visited_status=(QAStatus.NOT_VIEWED, QAStatus.VIEWED),
        )
        # TODO[Medium](Refactor): extract status_bar style to params
        self.status_bar = StatusBar(
            self._status_provider,
            self._pagination,
            {
                QAStatus.ADDED: QAStatus.ADDED.color,
                QAStatus.REMOVED: QAStatus.REMOVED.color,
                QAStatus.VIEWED: QAStatus.VIEWED.color,
                QAStatus.NOT_VIEWED: QAStatus.NOT_VIEWED.color,
            },  # type: ignore
        )
        self.pagination_widget = PaginationWidget(self._pagination, self._step_sizes)
        self.image_name_widget = widgets.Text(
            value="", description="", disabled=True, layout=widgets.Layout(width="90%")
        )
        self.image_container = widgets.HBox(
            layout=widgets.Layout(width="90%", border="2px solid black")
        )

    def _attach_callbacks(self):
        self._pagination.observe_index_changed(self._cb_change_image)

    def _cb_change_image(self, *args):
        self.update_image_box()
        self.update_image_container()

    def update_image_box(self):
        self.image_name_widget.value = f"[{self._pagination.current_index+1}] Image: {self._pagination.current_key}"

    def update_image_container(self):
        # Display the main image
        with open(self._pagination.current_key, "rb") as f:
            main_image = widgets.Image(
                value=f.read(),
                format="png",
                # width=960,
                # height=540,
                layout=widgets.Layout(width="100%"),
            )
            self.image_container.children = [main_image]

    def widgets(self, isolated=True) -> widgets.HBox | dict[str, widgets.Widget]:
        self._cb_change_image()
        if isolated:
            return widgets.VBox(
                [
                    self.image_name_widget,
                    self.image_container,
                    self.status_bar.widgets(isolated=True),
                    self.change_status_widget.widgets(isolated=True),
                    self.pagination_widget.widgets(isolated=True),
                ],
                layout=widgets.Layout(width="100%"),
            )

#### Example: ImageBrowser


In [35]:
image_files = list(list_paths("../../dev_notebooks/data/images/", r".*\.png"))
status_provider = DictStatusProvider(
    {p: QAStatus.NOT_VIEWED for p in image_files}, list(QAStatus)
)

ib = ImageBrowser(image_files, status_provider, start_index=0)

In [36]:
# | output: true


display(ib.widgets(isolated=True))

---


In [37]:
# | hide
from nbdev.showdoc import *

In [38]:
# | hide
import nbdev

nbdev.nbdev_export()